In [ ]:
# Personally I had to add the root folder of the repo to the sys.path.  If certain imports do not work you should uncomment and set the following.
# import sys
# sys.path.append('/root/of/repo/folder/')

# The Effects of a limited scope

The main reason for the introduction of different strategies is because of an issue, noted by previous works, where due to a large content churn and small population per edge node, reactive caching becomes unfeasible on smaller nodes. 

In this experiment we evaluate 4 different setups of edge nodes while keeping the number of users equal.  This means that in setups with more nodes the amount of users per node will decrease as will the number of requests.  We will use the LRU and Belady strategies to prove that this is indeed a problem.  To improve our confidence in the results we only plot the results after 10 runs with a confidence interval based on the standard deviation.

In [ ]:
from .utils import load_or_generate_trace, read_resource_map, setup_nodes, setup_stats_file_writers, make_dir, read_node_map, TraceIteratorProxy, calc_variance
from simulation.evaluator.strategy.runner import StrategyRunner
from simulation.evaluator.strategy.lru import LRUStrategy
from simulation.evaluator.statistics.cache_metrics import CacheMetrics

resource_file = "../dataset/out/dataset-resources-stats.csv"
pagemap_file = "../dataset/out/page-map-clean.csv"

node_maps = { 
    1: read_node_map('./node_setups/1node.json'), 
    3: read_node_map('./node_setups/3nodes.json'), 
    8: read_node_map('./node_setups/8nodes.json'), 
    14: read_node_map('./node_setups/14nodes.json') 
}
out_dir = make_dir('./out/experiment-limited-scope/')
lru_out_dir =  make_dir(f"{out_dir}lru/")
belady_out_dir = make_dir(f"{out_dir}beladys/")
no_users = 5000
no_iterations = 1000

## Traces
We generate our traces using the largest `node_map`.  Then later on we use a `TraceIteratorProxy` to make sure that we map the right nodes to match the smaller edge setups.  The traces are generated using 5000 users over 1000 iterations for three different generators: _zipf of 0.75_, _zipf of 1.3_ and _page map_.

In [ ]:
from simulation.generator.main_zipf import TraceConfig, Simulation
from simulation.generator.main_page_map import UserTraceConfig, UserSimulation

def generate_zipf_trace(seed: str, zipf_exponent: float):
    trace_config = TraceConfig(node_map=node_maps[14], seed=seed, no_users=no_users, no_iterations=no_iterations, zipf_exponent=zipf_exponent)
    simulation = Simulation(trace_config, resource_file)
    return load_or_generate_trace(f"{out_dir}/{trace_config.to_filename()}.trace.gz", simulation=simulation)

def generate_page_map_trace(seed: str):
    trace_config = UserTraceConfig(node_map=node_maps[14], seed=seed, no_users=no_users, no_iterations=no_iterations)
    user_simulation = UserSimulation(trace_config, pagemap_file)
    return load_or_generate_trace(f"{out_dir}/{trace_config.to_filename()}.trace.gz", simulation=user_simulation)

In [ ]:
no_runs = 10
trace_seeds = [ str(i) for i in range(no_runs) ]

## Proxy Setups
We set up three proxy maps to allow us to use the 1 node, 3 nodes, or 8 nodes setup with the 14 nodes traces.

In [ ]:
proxy_map = {
    1: { f"cdn{i + 1}": "cdn1" for i in range(14) },
    3: { "cdn1": "cdn1", "cdn2": "cdn2", "cdn3": "cdn3", "cdn4": "cdn2", "cdn5": "cdn2", "cdn6": "cdn1", "cdn7": "cdn3", "cdn8": "cdn3", "cdn9": "cdn1", "cdn10": "cdn2", "cdn11": "cdn1", "cdn12": "cdn3", "cdn13": "cdn3", "cdn14": "cdn3" },
    8: { "cdn1": "cdn1", "cdn2": "cdn2", "cdn3": "cdn3", "cdn4": "cdn4", "cdn5": "cdn5", "cdn6": "cdn6", "cdn7": "cdn7", "cdn8": "cdn8", "cdn9": "cdn5", "cdn10": "cdn4", "cdn11": "cdn1", "cdn12": "cdn8", "cdn13": "cdn7", "cdn14": "cdn3" },
    14: { f"cdn{i + 1}": f"cdn{i + 1}" for i in range(14) },
}

## LRU Setup

In [ ]:
def run_lru_experiment(trace, no_nodes, marker: str = "") -> dict[str, CacheMetrics]:
    trace_proxy = TraceIteratorProxy(trace.instructions, 
                                    proxy_map=proxy_map[no_nodes])
    nodes = setup_nodes(no_nodes, 1024 * 1024 * 1024 / no_nodes)
    stats_writers = setup_stats_file_writers(nodes, lru_out_dir, marker=f"n{no_nodes}-{marker}")
    strategy = LRUStrategy(nodes)
    return StrategyRunner(strategy, trace_proxy, read_resource_map(resource_file), stats_writers=stats_writers).perform()

## Belady's Setup

In [ ]:
from simulation.evaluator.strategy.belady_min import run_belady

def run_belady_experiment(trace, no_nodes, marker: str = "") -> dict[str, CacheMetrics]:
    trace_proxy = TraceIteratorProxy(trace.instructions, 
                                     proxy_map=proxy_map[no_nodes])
    cache_size = 1024*1024*1024 / no_nodes
    run_belady(trace_proxy, read_resource_map(resource_file), cache_size, belady_out_dir, marker=f"n{no_nodes}-{marker}")

## The Experiment

In [ ]:
print("Pre-generating Traces")
for seed in trace_seeds:
    generate_zipf_trace(seed=seed, zipf_exponent=0.75)
    generate_page_map_trace(seed=seed)
    generate_zipf_trace(seed=seed, zipf_exponent=1.30)
print("All traces generated")

In [ ]:
load_075_trace = lambda seed: (generate_zipf_trace(seed=seed, zipf_exponent=0.75), '075')
load_130_trace = lambda seed: (generate_zipf_trace(seed=seed, zipf_exponent=1.30), '130')
load_page_map_trace = lambda seed: (generate_page_map_trace(seed=seed), 'page-map')

trace_options = [ load_075_trace, load_130_trace, load_page_map_trace ]

def run_experiment(trace_seed: str, trace_loader, no_nodes: int):
    trace, trace_marker = trace_loader(trace_seed)
    print(trace_seed, trace_marker, no_nodes, 'STARTED')
    run_lru_experiment(trace, no_nodes, marker=f"{trace_marker}-{trace_seed}")
    run_belady_experiment(trace, no_nodes, marker=f"{trace_marker}-{trace_seed}")
    print(trace_seed, trace_marker, no_nodes, 'DONE')

In [ ]:
# Make use of multiprocess (over multiprocessing) if an "AttributeError" says it couldn't find `run_experiment`.
from multiprocessing import Pool

if __name__ == '__main__':
    options = [ (seed, trace, no_nodes) 
                for seed in trace_seeds
                for trace in trace_options 
                for no_nodes in node_maps.keys() ]
    print(f"Executing {len(options)} experiments...")
    with Pool() as p:
        p.starmap(run_experiment, options, chunksize=1)

## Plots
To visualise the results we create two plots that show the main performance metrics: hit ratio and backhaul cost.

But first we have to load in all the data from the runs above.

In [ ]:
from .utils import load_runs_in_dir, calc_ratio
import matplotlib.pyplot as plt
from palettable.colorbrewer.diverging import PuOr_4
puor_4 = PuOr_4.mpl_colors
puor_4[3] = PuOr_4.mpl_colors[2]
puor_4[2] = PuOr_4.mpl_colors[3]
import experiments.plotter.neat_plotter

belady_runs = load_runs_in_dir(belady_out_dir)
lru_runs = load_runs_in_dir(lru_out_dir)
belady_runs_decreasing = load_runs_in_dir(f"{out_dir}beladys-decreasing/")
lru_runs_decreasing = load_runs_in_dir(f"{out_dir}lru-decreasing/")

In [ ]:
def filter_runs_by(runs, match: str):
    return [ r for r in runs if match in str(r["source"]) ]

def calc_over_setups(runs, strategy: str, calculation) -> list[float]:
    setups = [ "-n1-", "-n3-", "-n8-", "-n14-" ]
    datapoints = []
    for setup in setups:
        filtered_runs = filter_runs_by(filter_runs_by(runs, setup), strategy)
        values = [ calculation(run) for run in filtered_runs ]
        datapoints.append(calc_variance(values))
    return datapoints

calc_average_hit_ratio = lambda run: calc_ratio(run['hits_total'][-1], run['misses_total'][-1])
calc_average_byte_ratio = lambda run: calc_ratio(run['cache_bytes_total'][-1], run['origin_bytes_total'][-1])

In [ ]:
from typing import Tuple
from .utils import generate_comparison_plot

def generate_plot(title: str, ylabel, strategies: dict[str, list[Tuple[float, float]]], linestyles: list[str] = None):
    x_labels = [ round(no_users / no_nodes, 1) for no_nodes in node_maps.keys()]
    plt.figure(num=None, figsize=(3, 4), dpi=300)
    generate_comparison_plot(plt, x_labels, strategies, colors=puor_4, linestyles=linestyles)
    plt.ylim(bottom=0, top=1.0)
    plt.ylabel(ylabel)
    plt.xlabel('Users per Node')
    plt.title(title)
    plt.xticks(x_labels, rotation=60)
    plt.legend()
    plt.show()

In [ ]:
generate_plot(title="Average Hit Ratios Zipf-0.75", ylabel='Hit Ratio', strategies={
    "Belady's": calc_over_setups(belady_runs, '-075-', calc_average_hit_ratio),
    "Belady's Decr. Storage": calc_over_setups(belady_runs_decreasing, '-075-', calc_average_hit_ratio),
    "LRU": calc_over_setups(lru_runs, '-075-', calc_average_hit_ratio),
    "LRU Decr. Storage": calc_over_setups(lru_runs_decreasing, '-075-', calc_average_hit_ratio)
}, linestyles=['solid', 'dashed', 'solid', 'dashed'])

In [ ]:
generate_plot(title="Average Hit Ratios Page-Map", ylabel='Hit Ratio', strategies={
    "Belady's": calc_over_setups(belady_runs, '-page-map-', calc_average_hit_ratio),
    "Belady's Decr. Storage": calc_over_setups(belady_runs_decreasing, '-page-map-', calc_average_hit_ratio),
    "LRU": calc_over_setups(lru_runs, '-page-map-', calc_average_hit_ratio),
    "LRU Decr. Storage": calc_over_setups(lru_runs_decreasing, '-page-map-', calc_average_hit_ratio),
}, linestyles=['solid', 'dashed', 'solid', 'dashed'])

In [ ]:
generate_plot(title="Average Hit Ratios Zipf-1.30", ylabel='Hit Ratio', strategies={
    "Belady's": calc_over_setups(belady_runs, '-130-', calc_average_hit_ratio),
    "Belady's Decr. Storage": calc_over_setups(belady_runs_decreasing, '-130-', calc_average_hit_ratio),
    "LRU": calc_over_setups(lru_runs, '-130-', calc_average_hit_ratio),
    "LRU Decr. Storage": calc_over_setups(lru_runs_decreasing, '-130-', calc_average_hit_ratio),
}, linestyles=['solid', 'dashed', 'solid', 'dashed'])

In [ ]:
generate_plot(title="Average Bandwidth Saved Zipf-0.75", ylabel='Fraction of Bandwidth Saved', strategies={
    "Belady's": calc_over_setups(belady_runs, '-075-', calc_average_byte_ratio),
    "Belady's Decr. Storage": calc_over_setups(belady_runs_decreasing, '-075-', calc_average_byte_ratio),
    "LRU": calc_over_setups(lru_runs, '-075-', calc_average_byte_ratio),
    "LRU Decr. Storage": calc_over_setups(lru_runs_decreasing, '-075-', calc_average_byte_ratio),
}, linestyles=['solid', 'dashed', 'solid', 'dashed'])

In [ ]:
generate_plot(title="Average Bandwidth Saved Page-Map", ylabel='Fraction of Bandwidth Saved', strategies={
    "Belady's": calc_over_setups(belady_runs, '-page-map-', calc_average_byte_ratio),
    "Belady's Decr. Storage": calc_over_setups(belady_runs_decreasing, '-page-map-', calc_average_byte_ratio),
    "LRU": calc_over_setups(lru_runs, '-page-map-', calc_average_byte_ratio),
    "LRU Decr. Storage": calc_over_setups(lru_runs_decreasing, '-page-map-', calc_average_byte_ratio),
}, linestyles=['solid', 'dashed', 'solid', 'dashed'])

In [ ]:
generate_plot(title="Average Bandwidth Saved Zipf-1.30", ylabel='Fraction of Bandwidth Saved', strategies={
    "Belady's": calc_over_setups(belady_runs, '-130-', calc_average_byte_ratio),
    "Belady's Decr. Storage": calc_over_setups(belady_runs_decreasing, '-130-', calc_average_byte_ratio),
    "LRU": calc_over_setups(lru_runs, '-130-', calc_average_byte_ratio),
    "LRU Decr. Storage": calc_over_setups(lru_runs_decreasing, '-130-', calc_average_byte_ratio),
}, linestyles=['solid', 'dashed', 'solid', 'dashed'])